In [1]:
import json
from datetime import datetime
from dateutil import tz
import pandas as pd
import matplotlib.pyplot as plt
import seaborn


In [2]:
with open('../newest_ts', 'r') as f:
    ts = f.read()

data_path = f"../data/data_{ts}.json"
with open(data_path, 'r') as f:
    data = json.load(f)

In [3]:
def ts2dt(ts, format='%Y-%m-%d %H:%M:%S'):
    return (datetime.utcfromtimestamp(int(ts))
                    .replace(tzinfo=tz.tzutc())
                    .astimezone(tz.tzlocal())
                    .strftime(format))

In [4]:
data.keys()

dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'current', 'hourly', 'daily'])

In [5]:
data['current']

{'dt': 1629644646,
 'sunrise': 1629583360,
 'sunset': 1629629370,
 'temp': 29.79,
 'feels_like': 36.79,
 'pressure': 1008,
 'humidity': 82,
 'dew_point': 26.38,
 'uvi': 0,
 'clouds': 25,
 'visibility': 10000,
 'wind_speed': 3.19,
 'wind_deg': 201,
 'wind_gust': 4.6,
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03n'}]}

In [6]:
[ts2dt(hrdt['dt'], format='%m-%d %H:%M') for hrdt in data['hourly']]

['08-22 15:00',
 '08-22 16:00',
 '08-22 17:00',
 '08-22 18:00',
 '08-22 19:00',
 '08-22 20:00',
 '08-22 21:00',
 '08-22 22:00',
 '08-22 23:00',
 '08-23 00:00',
 '08-23 01:00',
 '08-23 02:00',
 '08-23 03:00',
 '08-23 04:00',
 '08-23 05:00',
 '08-23 06:00',
 '08-23 07:00',
 '08-23 08:00',
 '08-23 09:00',
 '08-23 10:00',
 '08-23 11:00',
 '08-23 12:00',
 '08-23 13:00',
 '08-23 14:00',
 '08-23 15:00',
 '08-23 16:00',
 '08-23 17:00',
 '08-23 18:00',
 '08-23 19:00',
 '08-23 20:00',
 '08-23 21:00',
 '08-23 22:00',
 '08-23 23:00',
 '08-24 00:00',
 '08-24 01:00',
 '08-24 02:00',
 '08-24 03:00',
 '08-24 04:00',
 '08-24 05:00',
 '08-24 06:00',
 '08-24 07:00',
 '08-24 08:00',
 '08-24 09:00',
 '08-24 10:00',
 '08-24 11:00',
 '08-24 12:00',
 '08-24 13:00',
 '08-24 14:00']

In [7]:
data['hourly'][0]

{'dt': 1629644400,
 'temp': 29.79,
 'feels_like': 36.79,
 'pressure': 1008,
 'humidity': 82,
 'dew_point': 26.38,
 'uvi': 0,
 'clouds': 25,
 'visibility': 10000,
 'wind_speed': 3.19,
 'wind_deg': 201,
 'wind_gust': 4.6,
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03n'}],
 'pop': 0.08}

In [8]:
df_hourly = pd.DataFrame(data['hourly'])
df_daily = pd.DataFrame(data['daily'])

In [9]:
df_hourly.head()

,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,weather,pop,rain
0,1629644400,29.79,36.79,1008,82,26.38,0.0,25,10000,3.19,201,4.60,"[{'id': 802, 'main': 'Clouds', 'description': ...",0.08,NaN
1,1629648000,29.43,36.43,1008,83,26.24,0.0,28,10000,3.97,211,5.63,"[{'id': 802, 'main': 'Clouds', 'description': ...",0.00,NaN
2,1629651600,29.05,35.56,1008,83,25.87,0.0,35,10000,3.20,207,4.71,"[{'id': 802, 'main': 'Clouds', 'description': ...",0.04,NaN
3,1629655200,28.58,34.38,1007,84,25.61,0.0,45,10000,3.74,205,5.23,"[{'id': 500, 'main': 'Rain', 'description': 'l...",0.32,{'1h': 0.22}
4,1629658800,28.13,33.09,1007,84,25.17,0.0,85,10000,3.86,205,5.25,"[{'id': 500, 'main': 'Rain', 'description': 'l...",0.73,{'1h': 0.41}
